In [ ]:
import sys
sys.path.append('/Users/vdk/Software/code/')
import h5py
import pandas as pd
from astropy.table import Table, vstack
import numpy as np
from ctapipe.io import EventSource
from ctapipe.visualization import CameraDisplay
import matplotlib.pyplot as plt
import glob
from python_modules.muonpipe import usefull_func
import seaborn as sns
from ctapipe.image import tailcuts_clean
import scipy.stats as stats
colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#e377c2',  # raspberry yogurt pink
    '#8c564b',  # chestnut brown
    'k'
]

# Just to have subarray information for plotting purposes
filename = '/Users/vdk/muons2024/lapalma_simulations/simtel/zenith10/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 5000)


## Simulation processing

### LocalPeakWindowSum

Its a standard way how lstchain is processing muon rings in simulations. But for muons in the data lstchain uses GlobalPeakWindowSum

In [ ]:
local_fits_file = '/Users/vdk/muons2024/data/for_comparison/zenith10/local_peak_window_sum/muon_table_run1.fits'
local_dl1_file = '/Users/vdk/muons2024/data/for_comparison/zenith10/local_peak_window_sum/dl1_run101_muon.h5'

dat = Table.read(local_fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

df_sim_data_local = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

del(df_good_data)

local_sim_dl1_image = []
local_sim_dl1_peak_time = []
local_sim_dl1_image_mask = []
with h5py.File(local_dl1_file, 'r') as f:
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            local_sim_dl1_image.append(event_record[2])
            local_sim_dl1_peak_time.append(event_record[3])
            local_sim_dl1_image_mask.append(event_record[4])
        # This check is added in order to have the same statistic of same quality muon rings in all datasets
        if len(local_sim_dl1_image) == 542:
            break

### Visualisation of first couple of entries

In [ ]:
for i,event in enumerate(local_sim_dl1_image[:10]):
        plt.figure(figsize = (12,12))
        camgeom = source.subarray.tel[1].camera.geometry
        disp = CameraDisplay(camgeom,title='1')
        disp.image = event 
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        plt.show()

### GlobalPeakWindowSum

I specially used other config file with GlobalPeakWindowSum ImageExtractor for the same simtel file to check the difference between different charge extractors

In [ ]:
global_fits_file = '/Users/vdk/muons2024/data/for_comparison/zenith10/global_peak_window_sum/muon_table_run1.fits'
global_dl1_file = '/Users/vdk/muons2024/data/for_comparison/zenith10/global_peak_window_sum/dl1_run101_muon.h5'

dat = Table.read(global_fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

ring_completeness_cut = 0.8
ring_containment_cut = 0.99
event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

df_sim_data_global = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                           (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

global_sim_dl1_image = []
global_sim_dl1_peak_time = []
global_sim_dl1_image_mask = []
with h5py.File(global_dl1_file, 'r') as f:
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            global_sim_dl1_image.append(event_record[2])
            global_sim_dl1_peak_time.append(event_record[3])
            global_sim_dl1_image_mask.append(event_record[4])
        if len(global_sim_dl1_image) == 542:
            break


In [ ]:
for i,event in enumerate(global_sim_dl1_image[:10]):
        plt.figure(figsize = (12,12))
        camgeom = source.subarray.tel[1].camera.geometry
        disp = CameraDisplay(camgeom,title='1')
        disp.image = event
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        plt.show()

### NSB Tuned simulation (GlobalPeakWindowSum)

In [ ]:
fits_file = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/0.44nsb/muon_table_run1.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_sim_data_nsb1data_tuned = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

nsb_tuned_dl1_file = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/0.44nsb//dl1_run101_muon.h5'
nsb1data_tuned_sim_dl1_image = []
nsb1data_tuned_sim_dl1_peak_time = []
nsb1data_tuned_sim_dl1_image_mask = []
with h5py.File(nsb_tuned_dl1_file, 'r') as f:
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            nsb1data_tuned_sim_dl1_image.append(event_record[2])
            nsb1data_tuned_sim_dl1_peak_time.append(event_record[3])
            nsb1data_tuned_sim_dl1_image_mask.append(event_record[4])
        if len(nsb1data_tuned_sim_dl1_image) == 542:
            break



### NSB tuned on DL1

In [ ]:
fits_file = '/Users/vdk/muons2024/data/17043/nsb_tuning_dl1/muon_fits_tuned_2.fits'
dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_sim_data_nsb3data_tuned = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

nsb_tuned_dl1_file = '/Users/vdk/muons2024/data/17043/nsb_tuning_dl1/tuned.h5'
nsbDL1data_tuned_sim_dl1_image = []
nsbDL1data_tuned_sim_dl1_peak_time = []
nsbDL1data_tuned_sim_dl1_image_mask = []
with h5py.File(nsb_tuned_dl1_file, 'r') as f:
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            nsbDL1data_tuned_sim_dl1_image.append(event_record[2])
            nsbDL1data_tuned_sim_dl1_peak_time.append(event_record[3])
            nsbDL1data_tuned_sim_dl1_image_mask.append(event_record[4])
        if len(nsbDL1data_tuned_sim_dl1_image) == 542:
            break

#### NSB tuned but with LocalPeak

In [ ]:
fits_file = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/0.44nsb_local_peak/muon_table_run1.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_sim_data_nsb3data_tuned = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

nsb_tuned_local_peak_file = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/0.44nsb_local_peak/dl1_run101_muon.h5'
nsb_tuned_local_peak_dl1_image = []
nsb_tuned_local_peak_peak_time = []
nsb_tuned_local_peak_image_mask = []
with h5py.File(nsb_tuned_local_peak_file, 'r') as f:
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            nsb_tuned_local_peak_dl1_image.append(event_record[2])
            nsb_tuned_local_peak_peak_time.append(event_record[3])
            nsb_tuned_local_peak_image_mask.append(event_record[4])
        if len(nsb_tuned_local_peak_dl1_image)  == 542:
            break

### Mirror degraded relfection tuned sim

##### Nominal mirror degraded reflection is multiplied by 1.2

In [ ]:
fits_file = '/Users/vdk/muons2024/data/for_comparison/global_peak_window_sum/degraded_reflection_1.2/muon_table_run1.fits'


dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_sim_data_reflection_09408 = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))


relection_dl1_file = '/Users/vdk/muons2024/data/for_comparison/global_peak_window_sum/degraded_reflection_1.2/dl1_run101_muon.h5'
reflection09408_sim_dl1_image = []
reflection09408_sim_dl1_peak_time = []
reflection09408_sim_dl1_image_mask = []
with h5py.File(nsb_tuned_dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            reflection09408_sim_dl1_image.append(event_record[2])
            reflection09408_sim_dl1_peak_time.append(event_record[3])
            reflection09408_sim_dl1_image_mask.append(event_record[4])
        if len(reflection09408_sim_dl1_image) == 542:
            break

##### Nominal mirror degraded reflection is multiplied by 1.1

In [ ]:
fits_file = '/Users/vdk/muons2024/data/for_comparison/global_peak_window_sum/degraded_reflection_1.1/muon_table_run1.fits'


dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_sim_data_reflection_08624 = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

fits_file = '/Users/vdk/muons2024/data/for_comparison/global_peak_window_sum/degraded_reflection-9016/muon_table_run1.fits'


dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_sim_data_reflection_09016 = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))



### Processing of data

### Choose ID of muon events for further processing of .h5 files to get dl1 images

Its useful to do in such way, because .fits file are small, and h5 files take shitload amount of memory

In [ ]:
fits_file = '/Users/vdk/muons2024/data/20240310fits/muons_LST-1.Run17043.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

# This cuts value were choosen in order to get relatively good rings, and still have decent statistic out of one run, e.g. around 1k of events
ring_completeness_cut = 0.8
ring_containment_cut = 0.99
event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]))

# This is required for further cuts in the simulations
min_impact = min(df_good_data['impact_parameter'])
max_impact = max(df_good_data['impact_parameter']) 

filename_event_ids = '/Users/vdk/muons2024/data/17043_muons_08complet.txt'
#np.save(filename_event_ids, event_ids)

df_real_data = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]

df_real_data

Now based on choosen event id I choosed the required images on the cluster


TODO : add the link to the script which did it

In [ ]:
image_folder = glob.glob('/Users/vdk/muons2024/data/17043/dl1_image/*')
mask_folder = glob.glob('/Users/vdk/muons2024/data/17043/dl1_mask/*')
peak_folder = glob.glob('/Users/vdk/muons2024/data/17043/dl1_peak/*')

data_dl1_images = []
data_dl1_mask = []
data_dl1_peak = []

for file in image_folder:
    tmp_result = np.load(file)
    data_dl1_images.append(tmp_result)

for file in mask_folder:
    tmp_result = np.load(file)
    data_dl1_mask.append(tmp_result)

for file in peak_folder:
    tmp_result = np.load(file)
    data_dl1_peak.append(tmp_result)

#### Plots with images

In [ ]:
for i,event in enumerate(data_dl1_images[:20]):
        plt.figure(figsize = (12,12))
        camgeom = source.subarray.tel[1].camera.geometry
        disp = CameraDisplay(camgeom,title='1')
        disp.image = event 
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        plt.show()

#### Additional cleaning because for simulations in lstchain are used other thresholds

In [ ]:
new_data_mask = []
for image in data_dl1_images:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_data_mask.append(mask)
    
new_sim_mask = []
for image in nsb1data_tuned_sim_dl1_image:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_sim_mask.append(mask)

new_sim_mask_global = []
for image in global_sim_dl1_image:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_sim_mask_global.append(mask)

    
new_local_tune_subrun_mask = []
for image in nsb_tuned_local_peak_dl1_image:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_local_tune_subrun_mask.append(mask)

#### Creation of charge arrays with nsb and signal

In [ ]:
flattened_sim_local = np.array(local_sim_dl1_image).flatten()
flattened_sim_global = np.array(global_sim_dl1_image).flatten()
flattened_sim_nsb = np.array(nsb1data_tuned_sim_dl1_image[:2000]).flatten()
flattened_sim_nsb_local = np.array(nsb_tuned_local_peak_dl1_image).flatten()
flattened_data = np.array(data_dl1_images).flatten()
flattened_sim_nsbDL1data = np.array(nsbDL1data_tuned_sim_dl1_image[:600]).flatten()


data_masks_flat = np.array(new_data_mask).flatten()
global_sim_masks_flat = np.array(new_sim_mask_global).flatten()
nsb_tuned_sim_masks_flat = np.array(new_sim_mask).flatten()
nsb_tuned_local_sim_masks_flat = np.array(new_local_tune_subrun_mask).flatten()
nsbDL1data_tuned_sim_masks_flat = np.array(nsbDL1data_tuned_sim_dl1_image_mask).flatten()

# Separate pixel values based on the mask
data_pixels_with_mask_1 = flattened_data[data_masks_flat == 1]
data_pixels_with_mask_0 = flattened_data[data_masks_flat == 0]


global_sim_pixels_with_mask_1 = flattened_sim_global[global_sim_masks_flat == 1]
global_sim_pixels_with_mask_0 = flattened_sim_global[global_sim_masks_flat == 0]

nsb_tuned_sim_pixels_with_mask_1 = flattened_sim_nsb[nsb_tuned_sim_masks_flat == 1]
nsb_tuned_sim_pixels_with_mask_0 = flattened_sim_nsb[nsb_tuned_sim_masks_flat == 0] 

nsb_tuned_local_pixels_with_mask_1 = flattened_sim_nsb_local[nsb_tuned_local_sim_masks_flat == 1]
nsb_tuned_local_pixels_with_mask_0 = flattened_sim_nsb_local[nsb_tuned_local_sim_masks_flat == 0] 

nsbDL1data_tuned_sim_pixels_with_mask_1 = flattened_sim_nsbDL1data[nsbDL1data_tuned_sim_masks_flat == 1]
nsbDL1data_tuned_sim_pixels_with_mask_0 = flattened_sim_nsbDL1data[nsbDL1data_tuned_sim_masks_flat == 0]
   

#### Pixels with signal

In [ ]:
plt.figure(figsize=(16,11))

usefull_func.hist_wo_outliers(
    data_pixels_with_mask_1, 
    label = 'Data', 
    density=True, 
    mean_line=False, 
    show_outliers=True, 
    outlier_value=30,
    bins = 1855,
    color = colors[0], 
    alpha = 0.6
)
usefull_func.hist_wo_outliers(
    nsb_tuned_sim_pixels_with_mask_1, 
    label = 'Simulations with Tuned NSB', 
    density=True, 
    mean_line=False, 
    show_outliers=True, 
    outlier_value=30, 
    bins = 1855,
    color = colors[1], 
    alpha = 0.4
)
usefull_func.hist_wo_outliers(
    global_sim_pixels_with_mask_1, 
    label = 'Simulations With NSB 0.24GHz', 
    density=True, 
    mean_line=False, 
    show_outliers=True, 
    outlier_value=30, 
    bins = 1855,
    color = colors[2], 
    alpha = 0.4
)

plt.xlabel('Charge per pixel', fontsize = 14)
plt.ylabel('Probability density', fontsize = 14)
plt.ylim(0,0.5)
#plt.title("Charge per pixel only for pixels with signal")
plt.legend(prop={'size': 14})
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/dl1_hist_pixel_charge_signal.pdf', dpi=200, format='pdf', bbox_inches='tight')

#### Pixels with NSB

In [ ]:
plt.figure(figsize=(12,12))

usefull_func.hist_wo_outliers(
    data_pixels_with_mask_0, 
    label = 'Run17043', 
    density=True, 
    show_outliers=False, 
    outlier_value=30, 
    bins = 1855, 
    color = colors[0], 
    alpha = 0.6)

usefull_func.hist_wo_outliers(
    global_sim_pixels_with_mask_0, 
    label = 'Simulation with 0.24GHz NSB (standard value)', 
    density=True, 
    mean_line=False, 
    show_outliers=True, 
    outlier_value=30, 
    bins = 1855, 
    color = colors[6], 
    alpha = 0.5
)

usefull_func.hist_wo_outliers(
    nsb_tuned_sim_pixels_with_mask_0, 
    label = 'Simulations with NSB tuned on the waveforms', 
    density=True, 
    mean_line=False, 
    show_outliers=True, 
    outlier_value=30, 
    bins = 1855, 
    color = colors[1], 
    alpha = 0.6
)

usefull_func.hist_wo_outliers(
    nsbDL1data_tuned_sim_pixels_with_mask_0, 
    label = 'Simulations with NSB tuned on the DL1 level', 
    density=True, 
    mean_line=False, 
    show_outliers=True, 
    outlier_value=30, 
    bins = 1855, 
    color = colors[3], 
    alpha = 0.5
)

usefull_func.hist_wo_outliers(
    nsb_tuned_local_pixels_with_mask_0, 
    label = 'Tuned local peak window sum', 
    density=True, 
    mean_line=False, 
    show_outliers=True, 
    outlier_value=30, 
    bins = 1855, 
    color = colors[4], 
    alpha = 0.6
)


plt.xlabel('Charge per pixel', fontsize = 14)
plt.ylabel('Probability density', fontsize = 14)
plt.ylim(0,0.45)
#plt.title("Charge per pixel for pixels without signal")
plt.legend(prop={'size': 15})
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/dl1_hist_pixel_charge_no_signal_nsbtuned_clean.pdf', dpi=200, format='pdf', bbox_inches='tight')

#### Check how cleaning is choosing the pixels

In [ ]:
for test_image in data_dl1_images[:10]:
    mask = tailcuts_clean(camgeom, test_image, picture_thresh=10, boundary_thresh=5)
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    disp = CameraDisplay(
        camgeom, image=test_image, cmap="gray", ax = ax[0], title="Pixels with signal"
    )
    disp.highlight_pixels(mask, alpha=0.8, linewidth=2, color="green")
    disp = CameraDisplay(
        camgeom, image=test_image, cmap="gray", ax = ax[1], title="Pixels without signal"
    )
    disp.highlight_pixels(~mask, alpha=0.8, linewidth=2, color="green")

plt.close()

In [ ]:
for test_image in global_sim_dl1_image[:10]:
    mask = tailcuts_clean(camgeom, test_image, picture_thresh=10, boundary_thresh=5)
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    disp = CameraDisplay(
        camgeom, image=test_image, cmap="gray", ax = ax[0], title="Pixels with signal"
    )
    disp.highlight_pixels(mask, alpha=0.8, linewidth=2, color="green")
    disp = CameraDisplay(
        camgeom, image=test_image, cmap="gray", ax = ax[1], title="Pixels without signal"
    )
    disp.highlight_pixels(~mask, alpha=0.8, linewidth=2, color="green")

plt.close()

In [ ]:
for test_image in nsb1data_tuned_sim_dl1_image[:10]:
    mask = tailcuts_clean(camgeom, test_image, picture_thresh=10, boundary_thresh=5)
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    disp = CameraDisplay(
        camgeom, image=test_image, cmap="gray", ax = ax[0], title="Pixels with signal"
    )
    disp.highlight_pixels(mask, alpha=0.8, linewidth=2, color="green")
    disp = CameraDisplay(
        camgeom, image=test_image, cmap="gray", ax = ax[1], title="Pixels without signal"
    )
    disp.highlight_pixels(~mask, alpha=0.8, linewidth=2, color="green")

#### Visual comparison of how NSB level differs in DL1 images for data and sim

By setting the max amplitude of the pixel representation we can see in more details the brightness of particular pixels

In [ ]:
max_limit = 6
fig, ax = plt.subplots(1, 2, figsize=(18, 10))
camgeom = source.subarray.tel[1].camera.geometry
disp = CameraDisplay(
    camgeom, image=nsb1data_tuned_sim_dl1_image[2], cmap=plt.cm.RdBu_r, ax = ax[0], title="DL1 image of NSB tuned simulations"
)
#disp.set_limits_percent(20)
disp.set_limits_minmax(0,max_limit)
disp.add_colorbar()
disp = CameraDisplay(
    camgeom, image=data_dl1_images[2], cmap=plt.cm.RdBu_r, ax = ax[1], title="DL1 image of the data to which the NSB was tuned"
)

disp.add_colorbar()
disp.set_limits_minmax(0,max_limit)
#plt.savefig(f"/Users/vdk/TrurImageCheckCOG{i}_COG=0_fitWrong")
plt.show()